In [371]:
import requests
import urlparse
import os
import pickle
from bs4 import BeautifulSoup
import urllib2
import re
from datetime import datetime
from dateutil.parser import *

In [2]:
url = 'http://www.imdb.com/title/tt1430607/'
#http://www.imdb.com/search/title?genres=animation&title_type=feature&sort=moviemeter,asc

response = requests.get(url)

In [635]:
def get_imdb_soup(url):
    response = requests.get(url)
    page = response.text
    return BeautifulSoup(page) #for some reason cannot get attributes (ie.title,a,div) from prettify
    

In [626]:
numtitles = 4191

def iterurl(num_titles):
    i = 1
    urlist = ['http://www.imdb.com/search/title?genres=animation&sort=moviemeter,asc&start=1&title_type=feature']
    urlfront = "http://www.imdb.com/search/title?genres=animation&sort=moviemeter,asc&start="
    urlback = "&title_type=feature"
    while i < num_titles:
        i += 50
        url = urlfront+str(i)+urlback
        if i > (num_titles-40):
            break
        urlist.append(url)
    return urlist
        
imdb_base_url_list = iterurl(numtitles)
imdb_base_url_list

['http://www.imdb.com/search/title?genres=animation&sort=moviemeter,asc&start=1&title_type=feature',
 'http://www.imdb.com/search/title?genres=animation&sort=moviemeter,asc&start=51&title_type=feature',
 'http://www.imdb.com/search/title?genres=animation&sort=moviemeter,asc&start=101&title_type=feature',
 'http://www.imdb.com/search/title?genres=animation&sort=moviemeter,asc&start=151&title_type=feature',
 'http://www.imdb.com/search/title?genres=animation&sort=moviemeter,asc&start=201&title_type=feature',
 'http://www.imdb.com/search/title?genres=animation&sort=moviemeter,asc&start=251&title_type=feature',
 'http://www.imdb.com/search/title?genres=animation&sort=moviemeter,asc&start=301&title_type=feature',
 'http://www.imdb.com/search/title?genres=animation&sort=moviemeter,asc&start=351&title_type=feature',
 'http://www.imdb.com/search/title?genres=animation&sort=moviemeter,asc&start=401&title_type=feature',
 'http://www.imdb.com/search/title?genres=animation&sort=moviemeter,asc&star

In [636]:
base_url = 'http://www.imdb.com/search/title?genres=animation&sort=moviemeter,asc&start=1&title_type=feature'

In [641]:
def get_imdb_links(baseurl):
    basesoup = get_imdb_soup(baseurl)
    mvlinklist = []
    for a in basesoup.find_all('a'):
        if ('/title/tt' in a['href']) & (len(a['href']) == 17):
            mvlinklist.append(a['href'])
    return list(set(mvlinklist))

#get_imdb_links(base_url)


In [5]:
soup = BeautifulSoup(page)

In [6]:
#print soup.prettify()

In [7]:
#print soup.find("a")

In [8]:
#print soup.a.prettify()

In [9]:
for link in soup.find('link'):
    print link

In [10]:
soup.link

<link href="http://www.imdb.com/title/tt1430607/" rel="canonical"/>

In [422]:
url = 'http://www.imdb.com/title/tt2293640/'

In [424]:
def get_imdb_header(imdb_soup):
    imdb_header = imdb_soup.title
    return imdb_header


In [425]:
def get_imdb_title(imdb_header):
    header = imdb_header.text
    imdb_title = header.split('(')[0]
    return imdb_title

    
#get_imdb_title(imdb_header)  

In [426]:
def get_imdb_year(imdb_header):
    header = imdb_header.text
    year = re.findall(r'\d+', header)
    imdb_year = int(year[0])
    return imdb_year

    
#imdb_year(imdb_header)

In [427]:
def get_imdb_budget(soup):
        divs = [x.text for x in mv_soup.find_all('div')]
        for s in divs:
            start = 'Budget:'
            end = ' '
            result = re.findall('%s(.*)%s' % (start, end), s)
            for n in result:
                regex = re.compile(r'\d+(?:,\d+)*')
                bgt = re.findall(regex,n)
                budget =[x.replace(',','') for x in bgt]
                budint =[int(b) for b in budget]
        return budint[0]
                    
                

#get_imdb_budget(mv_soup)      

In [428]:
def get_imdb_openingwknd(soup):
        divs = [x.text for x in mv_soup.find_all('div')]
        for s in divs:
            start = 'Opening Weekend:'
            end = ' '
            result = re.findall('%s(.*)%s' % (start, end), s)
            for n in result:
                regex = re.compile(r'\d+(?:,\d+)*')
                opnwknd = re.findall(regex,n)
                openwknd =[x.replace(',','') for x in opnwknd]
                openingwknd =[int(b) for b in openwknd]
        return openingwknd[0]
                    
                

#get_imdb_openingwknd(mv_soup)      

In [429]:
def get_imdb_gross(soup):
        divs = [x.text for x in mv_soup.find_all('div')]
        for s in divs:
            start = 'Gross:'
            end = ' '
            result = re.findall('%s(.*)%s' % (start, end), s)
            for n in result:
                regex = re.compile(r'\d+(?:,\d+)*')
                grs = re.findall(regex,n)
                grss =[x.replace(',','') for x in grs]
                gross =[int(b) for b in grss]
        return gross[0]
                    
                
#get_imdb_gross(mv_soup)      

In [431]:
def get_imdb_runtime(soup):
    mvlen = soup.time
    mvtime = mvlen.text
    runtime = re.findall(r'\d+', mvtime)
    imdb_runtime = int(runtime[0])
    return imdb_runtime
    
                    
                
#get_imdb_runtime(mv_soup)   

91

In [417]:
def get_imdb_releasedate(soup):
    imdb_releasedate = soup.find(itemprop = 'datePublished')['content']
    #from dateutil.parse import *
    return parse(imdb_releasedate)

#get_imdb_releasedate(mv_soup)

datetime.datetime(2015, 6, 19, 0, 0)

In [527]:
def get_imdb_metascore(soup):
    a = soup.find_all('a')[90].text
    metascore = int(a.split('/')[0])
    return metascore
    
#get_imdb_metascore(mv_soup)

56

In [563]:
4191%50

41

'http://www.imdb.com/search/title?genres=animation&sort=moviemeter,asc&start=1&title_type=feature'

In [631]:
base_url = 'http://www.imdb.com/search/title?genres=animation&sort=moviemeter,asc&start=1&title_type=feature'

In [633]:
mwsp = get_imdb_soup(base_url)
mv_link_list = []
for a in mwsp.find_all('a'):
    if ('/title/tt' in a['href']) & (len(a['href']) == 17):
        mv_link_list.append(a['href'])
print list(set(mv_link_list))

['/title/tt0126029/', '/title/tt1911658/', '/title/tt2510894/', '/title/tt0107688/', '/title/tt2245084/', '/title/tt2294629/', '/title/tt0435761/', '/title/tt2267968/', '/title/tt1985949/', '/title/tt0398286/', '/title/tt0103639/', '/title/tt0110357/', '/title/tt1490017/', '/title/tt2452042/', '/title/tt0114709/', '/title/tt2096673/', '/title/tt3606756/', '/title/tt0837562/', '/title/tt1690953/', '/title/tt0119282/', '/title/tt0245429/', '/title/tt2974918/', '/title/tt0317705/', '/title/tt1979388/', '/title/tt2277860/', '/title/tt2224026/', '/title/tt0198781/', '/title/tt2262227/', '/title/tt0382932/', '/title/tt0114148/', '/title/tt1772341/', '/title/tt3819668/', '/title/tt2948356/', '/title/tt2709768/', '/title/tt1646971/', '/title/tt1754656/', '/title/tt2293640/', '/title/tt0061852/', '/title/tt1323594/', '/title/tt0120762/', '/title/tt0442933/', '/title/tt2141773/', '/title/tt0892769/', '/title/tt0266543/', '/title/tt0101414/', '/title/tt0372588/', '/title/tt2401878/', '/title/tt14

In [531]:
url = 'http://www.imdb.com/title/tt2096673/'

In [532]:
mv_soup = get_imdb_soup(url)
mv_header = get_imdb_header(mv_soup)
mv_title = get_imdb_title(mv_header)
mv_year = get_imdb_year(mv_header)
mv_budget = get_imdb_budget(mv_soup)
mv_openingwknd = get_imdb_openingwknd(mv_soup)
mv_gross = get_imdb_gross(mv_soup)
mv_runtime = get_imdb_runtime(mv_soup)
mv_releasedate = get_imdb_releasedate(mv_soup)
mv_metascore = get_imdb_metascore(mv_soup)

In [533]:
print mv_header
print mv_title
print mv_year
print mv_budget
print mv_openingwknd
print mv_gross
print mv_runtime
print mv_releasedate
print mv_metascore

<title>Inside Out (2015) - IMDb</title>
Inside Out 
2015
175000000
90440272
353495492
94
2015-06-19 00:00:00
94
